## Example on how to run the software

Run the first cell first to import modules and set paths


In [7]:
import os
from gnssmultipath import GNSS_MultipathAnalysis


base_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(base_path, os.pardir))

## Path to TestData
path_to_testdata = os.path.join(parent_dir, 'TestData')
ouputdir = os.path.join(parent_dir, 'Results')




#### Define the path to the RINEX observation file and the SP3 or navigation file


In [ ]:
rinObsFilename1 = os.path.join(path_to_testdata, 'ObservationFiles', 'OPEC00NOR_S_20220010000_01D_30S_MO_3.04.rnx')
broadcastNav4 = os.path.join(path_to_testdata, 'NavigationFiles', 'BRDC00IGS_R_20220010000_01D_MN.rnx')


#### Run the software with mandatory arguments only


In [ ]:
analysisResults = GNSS_MultipathAnalysis(rinObsFilename1,
                                         broadcastNav1=broadcastNav4)

####  Advanced example (more user defined settings)

In [ ]:
## Parameters
GNSSsystems                 = ["R"] # run analysis in GLONASS only
phaseCodeLimit              = 6.667
ionLimit                    = 0.0667
cutoff_elevation_angle      = 10 # 10 degree elevation cutoff
outputDir                   = ouputdir
plotEstimates               = True
plot_polarplot              = True
includeResultSummary        = True
includeCompactSummary       = True
includeObservationOverview  = True
includeLLIOverview          = True


## Rinex observation file
rinObsFilename1 = path_to_testdata  + '/ObservationFiles/' + 'OPEC00NOR_S_20220010000_01D_30S_MO_3.04.rnx'
## SP3 files
sp3NavFilename_1_opec = path_to_testdata  + '/SP3/' + 'Testfile_20220101.eph'

analysisResults = GNSS_MultipathAnalysis(rinObsFilename1,
                       desiredGNSSsystems=GNSSsystems,
                       sp3NavFilename_1 = sp3NavFilename_1_opec,
                       phaseCodeLimit = phaseCodeLimit,
                       ionLimit = ionLimit,
                       cutoff_elevation_angle = cutoff_elevation_angle,
                       outputDir = outputDir,
                       plotEstimates = plotEstimates,
                       plot_polarplot=plot_polarplot,
                       includeResultSummary = includeResultSummary,
                       includeCompactSummary = includeCompactSummary,
                       includeObservationOverview = includeObservationOverview,
                       includeLLIOverview = includeLLIOverview
                       )

How to read in a uncompressed result file in the pickle format

In [ ]:
# How to read in the result file from a analysis (UNCOMPRESSED)
from gnssmultipath import PickleHandler
path_to_resFile = os.path.join(ouputdir, 'analysisResults.pkl')
loaded_dictionary = PickleHandler.read_pickle(path_to_resFile)

How to read in a compressed result file in the pickle format

In [ ]:
# How to read in the result file from a analysis (COMPRESSED)
from gnssmultipath import PickleHandler
path_to_resFile = os.path.join(ouputdir, 'analysisResults.pkl')
loaded_dictionary = PickleHandler.read_zstd_pickle(path_to_resFile)

How to read a RINEX observation file

In [ ]:
from gnssmultipath import readRinexObs
GNSS_obs, GNSS_LLI, GNSS_SS, GNSS_SVs, time_epochs, nepochs, GNSSsystems,\
        obsCodes, approxPosition, max_sat, tInterval, markerName, rinexVersion, recType, timeSystem, leapSec, gnssType,\
        rinexProgr, rinexDate, antDelta, tFirstObs, tLastObs, clockOffsetsON, GLO_Slot2ChannelMap, success = readRinexObs(rinObsFilename1)


How to read a RINEX navigation file

In [ ]:
from gnssmultipath import Rinex_v3_Reader

navdata = Rinex_v3_Reader().read_rinex_nav(broadcastNav4, data_rate = 120)

In [ ]:
from gnssmultipath import GNSS_MultipathAnalysis

outputdir = r"/workspaces/GNSS_Multipath_Analysis_Software/Output"
rinObs = r"/workspaces/GNSS_Multipath_Analysis_Software/TestData/ObservationFiles/OPEC00NOR_S_20220010000_01D_30S_MO_3.04_croped.rnx"
rinNav1 = r"/workspaces/GNSS_Multipath_Analysis_Software/TestData/NavigationFiles/OPEC00NOR_S_20220010000_01D_CN.rnx"
rinNav2 = r"/workspaces/GNSS_Multipath_Analysis_Software/TestData/NavigationFiles/OPEC00NOR_S_20220010000_01D_EN.rnx"
rinNav3 = r"/workspaces/GNSS_Multipath_Analysis_Software/TestData/NavigationFiles/OPEC00NOR_S_20220010000_01D_GN.rnx"
rinNav4 = r"/workspaces/GNSS_Multipath_Analysis_Software/TestData/NavigationFiles/OPEC00NOR_S_20220010000_01D_RN.rnx"
analysisResults = GNSS_MultipathAnalysis(rinObs,
                                         broadcastNav1=rinNav1,
                                         broadcastNav2=rinNav2,
                                         broadcastNav3=rinNav3,
                                         broadcastNav4=rinNav4,
                                         plotEstimates=False,
                                         save_results_as_compressed_pickle=True,
                                         outputDir=outputdir)

### Estimate approximate receiver posistion based on pseudoranges

#### Use a SP3 file

In [ ]:
from gnssmultipath import GNSSPositionEstimator
import numpy as np

rinObs = os.path.join(path_to_testdata, 'ObservationFiles/OPEC00NOR_S_20220010000_01D_30S_MO_3.04_croped.rnx')
sp3 = os.path.join(path_to_testdata, 'SP3/Testfile_20220101.eph')

# Set desired time for when to estimate position and which system to use
desired_time = np.array([2022, 1, 1, 1, 5, 30.0000000])
desired_system = "G"  # GPS
gnsspos, stats = GNSSPositionEstimator(rinObs,
                                    sp3_file = sp3,
                                    desired_time = desired_time,
                                    desired_system = desired_system,
                                    elevation_cut_off_angle = 15
                                    ).estimate_position()

print('Estimated coordinates in ECEF (m):\n' + '\n'.join([f'{axis} = {coord}' for axis, coord in zip(['X', 'Y', 'Z'], np.round(gnsspos[:-1], 3))]))
print('\nStandard deviation of the estimated coordinates (m):\n' + '\n'.join([f'{k} = {v}' for k, v in stats["Standard Deviations"].items() if k in ['Sx', 'Sy', 'Sz']]))

#### Use a RINEX navigation file and print DOP values

In [ ]:
from gnssmultipath import GNSSPositionEstimator
import numpy as np

rinObs = os.path.join(path_to_testdata, 'ObservationFiles/OPEC00NOR_S_20220010000_01D_30S_MO_3.04_croped.rnx')
rinNav = os.path.join(path_to_testdata, 'NavigationFiles/BRDC00IGS_R_20220010000_01D_MN.rnx')

# Set desired time for when to estimate position and which system to use
desired_time = np.array([2022, 1, 1, 2, 40, 0.0000000])
desired_system = "R"  # GLONASS


gnsspos, stats = GNSSPositionEstimator(rinObs,
                                    rinex_nav_file = rinNav,
                                    desired_time = desired_time,
                                    desired_system = desired_system,
                                    elevation_cut_off_angle = 10).estimate_position()

print('Estimated coordinates in ECEF (m):\n' + '\n'.join([f'{axis} = {coord}' for axis, coord in zip(['X', 'Y', 'Z'], np.round(gnsspos[:-1], 3))]))
print('\nStandard deviation of the estimated coordinates (m):\n' + '\n'.join([f'{k} = {v}' for k, v in stats["Standard Deviations"].items() if k in ['Sx', 'Sy', 'Sz']]))
print(f'\nDOP values:\n' + '\n'.join([f'{k} = {v}' for k, v in stats["DOPs"].items()]))

### Get approximate recevier coordinates in user defined CRS

In [ ]:
from gnssmultipath import GNSSPositionEstimator
import numpy as np


rinObs = os.path.join(path_to_testdata, 'ObservationFiles/OPEC00NOR_S_20220010000_01D_30S_MO_3.04_croped.rnx')
rinNav = os.path.join(path_to_testdata, 'NavigationFiles/BRDC00IGS_R_20220010000_01D_MN.rnx')

# Set desired time for when to estimate position and which system to use
desired_time = np.array([2022, 1, 1, 1, 5, 30.0000000])
desired_system = "E"  # GPS
desired_crs = "EPSG:32632"  # Desired CRS on the estimated receiver coordiantes (WGS84 UTM zone 32N)
gnsspos, stats = GNSSPositionEstimator(rinObs,
                                    rinex_nav_file=rinNav,
                                    desired_time = desired_time,
                                    desired_system = desired_system,
                                    elevation_cut_off_angle = 10,
                                    crs=desired_crs).estimate_position()

print('Estimated coordinates in ECEF (m):\n' + '\n'.join([f'{axis} = {coord}' for axis, coord in zip(['Eastin', 'Northing', 'Height (ellipoidal)'], np.round(gnsspos[:-1], 3))]))